# CODE QUI DOIT FONCTIONNER

In [1]:
import os
import pandas as pd
import networkx as nx
from pyvis.network import Network
from IPython.display import display, HTML
import ast
import json

In [40]:
hierarchical_topics = pd.read_csv(f"archive/test/result/database_hierarchical_topics.csv")
news_data = pd.read_csv(f"archive/test/result/database_update.csv")
all_topic = pd.read_csv(f"archive/test/result/all_topic.csv")


news_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", select_menu=True, layout=False)    
news_net.barnes_hut()
max_original_topic = hierarchical_topics.Parent_ID.astype(int).min() - 1

        # Extract mapping from ID to name
topic_to_name = dict(zip(hierarchical_topics.Child_Left_ID, hierarchical_topics.Child_Left_Name))
topic_to_name.update(dict(zip(hierarchical_topics.Child_Right_ID, hierarchical_topics.Child_Right_Name)))
topic_to_name.update(dict(zip(hierarchical_topics.Parent_ID, hierarchical_topics.Parent_Name)))

topic_to_name = {topic: name for topic, name in topic_to_name.items()}
#print((topic_to_name))
        # Create tree
tree = {row[1].Parent_ID: [row[1].Child_Left_ID, row[1].Child_Right_ID]
        for row in hierarchical_topics.iterrows()}


for parent,childs in tree.items():
        src = parent
        src_names = topic_to_name[parent]
        lvl = hierarchical_topics.loc[hierarchical_topics['Parent_ID'] == parent, 'Distance'].values[0]
        news_net.add_node(src, src_names, title=src,level=src)
        for child in childs:
                dst = child
                dst_names = topic_to_name[child]
                news_net.add_node(dst, dst_names, title=dst,level=dst)
                news_net.add_edge(src, dst,values=lvl)
        topics = hierarchical_topics.loc[hierarchical_topics['Parent_ID'] == parent, 'Topics'].values
        filter_child = [element for element in topics if element not in childs]
        for child in filter_child:
                dst = child
                dst_names = all_topic.loc[all_topic['Topic'] == parent, 'Name'].values[0]
                news_net.add_node(dst, dst_names, title=dst)
                news_net.add_edge(src, dst,values=lvl)
           

list_topics = all_topic['Topic']

for topic in list_topics:
        list_doc = news_data.loc[news_data['topic'] == topic,'id' 'title',"keyword"].values
        print(list_topics)
        for id,title,keywords in list_doc :
                news_net.add_node(id,title,title=id)
                news_net.add_edge(topic,id)
                for keyword,distance in keywords :
                        max = news_net.num_nodes()+1
                        news_net.add_nodes(max,keyword,title=max)
                        news_net.add_edge(id,max,distance)


neighbor_map = news_net.get_adj_list()
min_level = hierarchical_topics["Parent_ID"].min()-1
for node in news_net.nodes:
# Assurez-vous que node["title"] est une chaîne de caractères
        node_title_str = node["label"]
        neighbors_str = ", ".join(neighbor["label"] for neighbor in neighbor_map[node["id"]])
        # Utilisez la version chaîne de caractères de node["title"] pour la concaténation
        node["title"] = node_title_str + "\n Neighbors : [" + neighbors_str + "]"
        node["value"] = len(neighbor_map[node["id"]])
        if len(neighbor_map[node["id"]])<2:
                node["level"]=int(min_level)




news_net.toggle_physics(True)
news_net.show_buttons(True)
news_net.inherit_edge_colors(True)
news_net.save_graph("graphs_.html")   

# Tree test

In [17]:
max_distance = None
width = 1 
hier_topics = pd.read_csv(f"archive/test/result/database_hierarchical_topics.csv")
if max_distance is None:
    max_distance = hier_topics.Distance.max() + 1

max_original_topic = hier_topics.Parent_ID.astype(int).min() - 1

# Extract mapping from ID to name
topic_to_name = dict(zip(hier_topics.Child_Left_ID, hier_topics.Child_Left_Name))
topic_to_name.update(dict(zip(hier_topics.Child_Right_ID, hier_topics.Child_Right_Name)))
topic_to_name = {str(topic): name[:100] for topic, name in topic_to_name.items()}

# Create tree
tree = {str(row[1].Parent_ID): [str(row[1].Child_Left_ID), str(row[1].Child_Right_ID)]
        for row in hier_topics.iterrows()}

def get_tree(start, tree):
    """ Based on: https://stackoverflow.com/a/51920869/10532563 """

    def _tree(to_print, start, parent, tree, grandpa=None, indent=""):

        # Get distance between merged topics
        distance = hier_topics.loc[(hier_topics.Child_Left_ID == parent) |
                                    (hier_topics.Child_Right_ID == parent), "Distance"]
        distance = distance.values[0] if len(distance) > 0 else 10

        if parent != start:
            if grandpa is None:
                to_print += topic_to_name[parent]
            else:
                if int(parent) <= max_original_topic:
                    to_print += "■──" + topic_to_name[parent] + f" ── Topic: {parent}" + "\n"
                    
                else:
                    to_print += topic_to_name[parent] + "\n"

        if parent not in tree:
            return to_print

        for child in tree[parent][:-1]:
            to_print += indent + "├" + "─"
            to_print = _tree(to_print, start, child, tree, parent, indent + "│" + " " * width)

        child = tree[parent][-1]
        to_print += indent + "└" + "─"
        to_print = _tree(to_print, start, child, tree, parent, indent + " " * (width+1))

        return to_print

    to_print = "." + "\n"
    to_print = _tree(to_print, start, start, tree)
    return to_print

start = str(hier_topics.Parent_ID.astype(int).max())
print(get_tree(start, tree))

.
├─■──griffin_kathy_photo_comedian_cnn ── Topic: 180
└─said_mr_would_trump_one
  ├─■──ubisoft_players_e3_game_multiplayer ── Topic: 162
  └─said_mr_would_trump_one
    ├─said_mr_would_trump_one
    │ ├─said_mr_would_trump_one
    │ │ ├─■──dogs_dog_pape_pet_bailey ── Topic: 136
    │ │ └─said_mr_would_trump_one
    │ │   ├─said_mr_would_trump_one
    │ │   │ ├─said_mr_would_trump_one
    │ │   │ │ ├─■──christie_wildstein_governor_baroni_closings ── Topic: 91
    │ │   │ │ └─said_mr_would_trump_one
    │ │   │ │   ├─said_mr_trump_would_one
    │ │   │ │   │ ├─■──hodgkinson_scalise_shooting_steve_rep ── Topic: 191
    │ │   │ │   │ └─said_mr_would_trump_one
    │ │   │ │   │   ├─■──breitbart_siriusxm_news_daily_discuss ── Topic: 85
    │ │   │ │   │   └─said_mr_would_trump_one
    │ │   │ │   │     ├─said_mr_would_trump_one
    │ │   │ │   │     │ ├─said_mr_would_trump_one
    │ │   │ │   │     │ │ ├─■──neanderthals_fossils_neanderthal_finlayson_gibraltar ── Topic: 152
    │ │   │ │   │ 

In [6]:
import pandas as pd

df = pd.read_csv(f"../data/archive/database_merge.csv")
key_to_check = 7174
print(df[7170:7180])
print(df.index[7170:7180])
if key_to_check in df.index:
    # Proceed with operations on 
    print("Key exists!")
else:
    print("Key does not exist in the DataFrame index.")


           id                                              title  \
7170  25698.0  Without Obama, Shinzo Abe’s Approach to U.S.-J...   
7171  25700.0  Giving Up ‘Mostly Everything’ to Care for His ...   
7172  25701.0  Carrie Fisher, Trump, Israel: Your Wednesday E...   
7173  25703.0  How to Have a Dinner Party: Friends Not Requir...   
7174  25704.0                   The Fighter - The New York Times   
7175  25705.0  A Brother’s Crimes and a Sister’s Betrayal Mak...   
7176  25708.0  Uncertainty Over New Chinese Law Rattles Forei...   
7177  25709.0  A Majority Agreed She Was Raped by a Stanford ...   
7178  25710.0  Searching for the Human Factor in Deadly Avala...   
7179  25713.0  Michael Skakel’s Murder Conviction Has Been Re...   

         publication                             author        date    year  \
7170  New York Times                        Motoko Rich  2017-01-05  2017.0   
7171  New York Times                          John Otis  2016-12-29  2016.0   
7172  New York